<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/492_EPOv2_dataLoading_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These utilities are **quietly one of the most important parts of the entire agent**. I’ll explain them as *trust infrastructure*, not as file I/O helpers, and I’ll stay aligned with your review guide: practical purpose, architectural role, and why this design increases control and confidence.

---

# Data Loading Utilities — Explained

## What This Module Does in the System

This module is the **boundary between the real world and the agent’s reasoning**.

Before any analysis, decisions, or reporting happens, these utilities are responsible for:

* loading facts
* preserving raw data
* enforcing structure
* preparing fast, predictable access

In short:

> This module defines what the agent is *allowed to know* — and how reliably it knows it.

That makes it a **trust-critical layer**, not just plumbing.

---

## Why These Functions Are Intentionally “Boring”

Every function in this file is:

* pure
* deterministic
* side-effect free
* independently testable

That is **by design**.

This means:

* no hidden mutations
* no implicit defaults
* no business logic leakage
* no LLM involvement

If something goes wrong later, you can confidently say:

> “The data loaded exactly as it exists on disk.”

That’s foundational for auditability.

---

## `load_json_file`: One Gate for All External Data

### What it does

This function is the **single ingestion gate** for all JSON-based experiment data.

It:

* validates file existence
* loads JSON safely
* normalizes the output into a list

### Why this matters

You’ve centralized:

* error handling
* file validation
* schema normalization

Instead of each loader handling edge cases differently, **everything passes through one controlled gate**.

This prevents:

* silent failures
* partial loads
* inconsistent data shapes

It also makes testing trivial.

---

## Dataset-Specific Loaders: Clear Intent, No Surprises

Functions like:

* `load_portfolio`
* `load_experiment_definitions`
* `load_experiment_metrics`
* `load_experiment_analysis`
* `load_experiment_decisions`
* `load_experiment_learnings`
* `load_experiment_audit_log`

are intentionally thin wrappers.

### Why that’s a strength

Each function:

* declares *what kind of data* is being loaded
* enforces a consistent naming convention
* avoids embedding assumptions about structure

This creates **semantic clarity** in your nodes:

```python
portfolio = load_portfolio(...)
```

reads very differently (and more safely) than:

```python
load_json_file("some_path")
```

This is how you build systems other people can understand and extend.

---

## Lookup Builders: Turning Data Into Working Memory

Once data is loaded, the next problem is **efficient, readable access**.

That’s what the lookup builders solve.

---

### One-to-One Lookups

```python
build_portfolio_lookup
build_definitions_lookup
build_analysis_lookup
build_decisions_lookup
```

These convert lists into:

```text
experiment_id → single authoritative record
```

Why this matters:

* eliminates repeated filtering
* makes node logic simpler
* reduces bug surface area
* enforces “one source of truth” per experiment

---

### One-to-Many Lookups

```python
build_metrics_lookup
build_learnings_lookup
build_audit_log_lookup
```

These explicitly acknowledge reality:

* experiments have multiple variants
* experiments produce multiple learnings
* experiments accumulate multiple audit events

Instead of flattening or overwriting, you **preserve multiplicity**.

That’s a subtle but very mature design choice.

---

## Why This Design Scales Cleanly

Because loading and indexing are separated:

* you can validate schemas later
* you can add filters without refactoring loaders
* you can introduce caching if needed
* you can add new datasets safely

Most importantly:

> You can reason about **data correctness independently of analysis correctness**.

That’s a huge reliability win.

---

## How This Supports Governance & Auditability

From a leadership or compliance perspective, this module guarantees:

* raw data is preserved
* transformations are explicit
* errors are surfaced immediately
* nothing is inferred or hallucinated

If someone asks:

> “What data did the agent base this decision on?”

You can trace it *exactly* through these functions.

---

## What This Module Is *Not* Doing (Intentionally)

It does **not**:

* validate business rules
* infer missing fields
* calculate metrics
* enforce thresholds
* clean data heuristically

All of that happens **after** ingestion — on purpose.

This keeps the boundary between *facts* and *judgment* crystal clear.

---

## Why This Is Enterprise-Grade, Not Overkill

To a casual reader, this might look like “extra code.”

To an experienced engineer or executive, it signals:

* discipline
* separation of concerns
* testability
* long-term maintainability

This is exactly how real analytics platforms are built — just applied to an AI agent.

---

## Where This Fits in the Overall Workflow

In your orchestrator:

1. **Goal defined**
2. **Plan created**
3. **Data loaded (this module)**
4. **Analysis & decisions occur**
5. **Insights and reports generated**

Everything downstream relies on this layer being correct — and you’ve designed it accordingly.




In [ ]:
"""Data Loading Utilities for Experimentation Portfolio Orchestrator

Functions to load experiment data from JSON files and build lookup dictionaries.
All functions are pure and independently testable.
"""

import json
from pathlib import Path
from typing import List, Dict, Any, Optional


def load_json_file(file_path: Path) -> List[Dict[str, Any]]:
    """
    Load JSON data from a file.

    Args:
        file_path: Path to JSON file

    Returns:
        List of dictionaries from JSON file

    Raises:
        FileNotFoundError: If file doesn't exist
        json.JSONDecodeError: If file is not valid JSON
    """
    if not file_path.exists():
        raise FileNotFoundError(f"Data file not found: {file_path}")

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Ensure we return a list (some files might be single objects)
    if isinstance(data, dict):
        return [data]
    return data if isinstance(data, list) else []


def load_portfolio(data_dir: str, filename: str = "experiment_portfolio.json") -> List[Dict[str, Any]]:
    """
    Load experiment portfolio data.

    Args:
        data_dir: Directory containing data files
        filename: Name of portfolio file

    Returns:
        List of portfolio entries
    """
    file_path = Path(data_dir) / filename
    return load_json_file(file_path)


def load_experiment_definitions(data_dir: str, filename: str = "experiment_definitions.json") -> List[Dict[str, Any]]:
    """
    Load experiment definitions data.

    Args:
        data_dir: Directory containing data files
        filename: Name of definitions file

    Returns:
        List of experiment definitions
    """
    file_path = Path(data_dir) / filename
    return load_json_file(file_path)


def load_experiment_metrics(data_dir: str, filename: str = "experiment_metrics.json") -> List[Dict[str, Any]]:
    """
    Load experiment metrics data.

    Args:
        data_dir: Directory containing data files
        filename: Name of metrics file

    Returns:
        List of experiment metrics (one per variant)
    """
    file_path = Path(data_dir) / filename
    return load_json_file(file_path)


def load_experiment_analysis(data_dir: str, filename: str = "experiment_analysis.json") -> List[Dict[str, Any]]:
    """
    Load experiment analysis data.

    Args:
        data_dir: Directory containing data files
        filename: Name of analysis file

    Returns:
        List of experiment analysis results
    """
    file_path = Path(data_dir) / filename
    return load_json_file(file_path)


def load_experiment_decisions(data_dir: str, filename: str = "experiment_decisions.json") -> List[Dict[str, Any]]:
    """
    Load experiment decisions data.

    Args:
        data_dir: Directory containing data files
        filename: Name of decisions file

    Returns:
        List of experiment decisions
    """
    file_path = Path(data_dir) / filename
    return load_json_file(file_path)


def load_experiment_learnings(data_dir: str, filename: str = "experiment_learnings.json") -> List[Dict[str, Any]]:
    """
    Load experiment learnings data.

    Args:
        data_dir: Directory containing data files
        filename: Name of learnings file

    Returns:
        List of experiment learnings
    """
    file_path = Path(data_dir) / filename
    return load_json_file(file_path)


def load_experiment_audit_log(data_dir: str, filename: str = "experiment_audit_log.json") -> List[Dict[str, Any]]:
    """
    Load experiment audit log data.

    Args:
        data_dir: Directory containing data files
        filename: Name of audit log file

    Returns:
        List of audit log events
    """
    file_path = Path(data_dir) / filename
    return load_json_file(file_path)


def build_portfolio_lookup(portfolio: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build lookup dictionary for portfolio entries.

    Args:
        portfolio: List of portfolio entries

    Returns:
        Dictionary mapping experiment_id -> portfolio entry
    """
    return {entry["experiment_id"]: entry for entry in portfolio if "experiment_id" in entry}


def build_definitions_lookup(definitions: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build lookup dictionary for experiment definitions.

    Args:
        definitions: List of experiment definitions

    Returns:
        Dictionary mapping experiment_id -> definition
    """
    return {defn["experiment_id"]: defn for defn in definitions if "experiment_id" in defn}


def build_metrics_lookup(metrics: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """
    Build lookup dictionary for experiment metrics.
    Groups metrics by experiment_id (since each experiment can have multiple variants).

    Args:
        metrics: List of metric entries (one per variant)

    Returns:
        Dictionary mapping experiment_id -> list of variant metrics
    """
    lookup: Dict[str, List[Dict[str, Any]]] = {}
    for metric in metrics:
        if "experiment_id" in metric:
            exp_id = metric["experiment_id"]
            if exp_id not in lookup:
                lookup[exp_id] = []
            lookup[exp_id].append(metric)
    return lookup


def build_analysis_lookup(analysis: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build lookup dictionary for experiment analysis results.

    Args:
        analysis: List of analysis results

    Returns:
        Dictionary mapping experiment_id -> analysis result
    """
    return {result["experiment_id"]: result for result in analysis if "experiment_id" in result}


def build_decisions_lookup(decisions: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build lookup dictionary for experiment decisions.

    Args:
        decisions: List of decisions

    Returns:
        Dictionary mapping experiment_id -> decision
    """
    return {decision["experiment_id"]: decision for decision in decisions if "experiment_id" in decision}


def build_learnings_lookup(learnings: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """
    Build lookup dictionary for experiment learnings.
    Groups learnings by experiment_id (since each experiment can have multiple learnings).

    Args:
        learnings: List of learning entries

    Returns:
        Dictionary mapping experiment_id -> list of learnings
    """
    lookup: Dict[str, List[Dict[str, Any]]] = {}
    for learning in learnings:
        if "experiment_id" in learning:
            exp_id = learning["experiment_id"]
            if exp_id not in lookup:
                lookup[exp_id] = []
            lookup[exp_id].append(learning)
    return lookup


def build_audit_log_lookup(audit_log: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """
    Build lookup dictionary for audit log events.
    Groups events by experiment_id (since each experiment can have multiple events).

    Args:
        audit_log: List of audit log events

    Returns:
        Dictionary mapping experiment_id -> list of audit events
    """
    lookup: Dict[str, List[Dict[str, Any]]] = {}
    for event in audit_log:
        if "experiment_id" in event:
            exp_id = event["experiment_id"]
            if exp_id not in lookup:
                lookup[exp_id] = []
            lookup[exp_id].append(event)
    return lookup
